In [2]:
import numpy as np
from scipy import stats
from scipy.stats import multivariate_normal
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import multivariate_normal as mvn

In [3]:
import csv

In [17]:
class GaussianMixtureModel():
    """Density estimation with Gaussian Mixture Models (GMM).

    You can add new functions if you find it useful, but **do not** change
    the names or argument lists of the functions provided.
    """
    def __init__(self, X, K):
        """Initialise GMM class.

        Arguments:
          X -- data, N x D array
          K -- number of mixture components, int
        """
        self.X = X
        self.n = X.shape[0]
        self.D = X.shape[1]
        self.K = K


    def E_step(self, mu, S, pi):
        """Compute the E step of the EM algorithm.

        Arguments:
          mu -- component means, K x D array
          S -- component covariances, K x D x D array
          pi -- component weights, K x 1 array

        Returns:
          r_new -- updated component responsabilities, N x K array
        """
        # Assert that all arguments have the right shape
        assert(mu.shape == (self.K, self.D) and\
               S.shape  == (self.K, self.D, self.D) and\
               pi.shape == (self.K, 1))
        r_new = np.zeros((self.n, self.K))

        # Task 1: implement the E step and return updated responsabilities
        # Write your code from here...

        for i in range(self.n): 
            Nk = 0
            for j in range(self.K):
                Nk += pi[j] * mvn.pdf((self.X)[i], mu[j], S[j], allow_singular=True)
            for k in range(self.K):
                r_new[i, k] = pi[k] * mvn.pdf((self.X)[i], mu[k], S[k], allow_singular=True)/ Nk
        
        # ... to here.
        assert(r_new.shape == (self.n, self.K))
        return r_new


    def M_step(self, mu, r):
        """Compute the M step of the EM algorithm.

        Arguments:
          mu -- previous component means, K x D array
          r -- previous component responsabilities,  N x K array

        Returns:
          mu_new -- updated component means, K x D array
          S_new -- updated component covariances, K x D x D array
          pi_new -- updated component weights, K x 1 array
        """
        assert(mu.shape == (self.K, self.D) and\
               r.shape  == (self.n, self.K))
        mu_new = np.zeros((self.K, self.D))
        S_new  = np.zeros((self.K, self.D, self.D))
        pi_new = np.zeros((self.K, 1))

        # Task 2: implement the M step and return updated mixture parameters
        # Write your code from here...

        #updating mu and pi
        for k in range(self.K):
            Nk = 0
            for j in range(self.n):
                Nk += r[j,k]
            for i in range(self.n):
                mu_new[k] += (r[i, k] * (self.X)[i])
            mu_new[k] /= Nk
            pi_new[k] = Nk/ (self.n)
        
        #updating S
        for k in range(self.K):
            Nk = 0
            for i in range(self.n): 
                Nk += r[i,k]
            for i in range(self.n):
                ys = np.reshape((self.X)[i]- mu_new[k], (-1,1))
                S_new[k] += (r[i, k] * (ys @ (ys.T)))
            S_new[k] /= Nk
        
        # ... to here.
        assert(mu_new.shape == (self.K, self.D) and\
               S_new.shape  == (self.K, self.D, self.D) and\
               pi_new.shape == (self.K, 1))
        return mu_new, S_new, pi_new

    
    def log_likelihood(self, pi, mu, S):
        '''Compute the loglikelihood'''
        
        ll = 0
        for i in range(self.n):
                s = 0
                for k in range(K):
                    s += pi[k] * mvn.pdf((self.X)[i], mu[k], S[k], allow_singular=True)
                ll += np.log(s)
        return -ll
    
    def train(self, initial_params):
        """Fit a Gaussian Mixture Model (GMM) to the data in matrix X.

        Arguments:
          initial_params -- dictionary with fields 'mu', 'S', 'pi' and 'K'

        Returns:kernel 
          mu -- component means, K x D array
          S -- component covariances, K x D x D array
          pi -- component weights, K x 1 array
          r -- component responsabilities, N x K array
        """
        # Assert that initial_params has all the necessary fields
        assert(all([k in initial_params for k in ['mu', 'S', 'pi']]))

        mu = np.zeros((self.K, self.D))
        S  = np.zeros((self.K, self.D, self.D))
        pi = np.zeros((self.K, 1))
        r  = np.zeros((self.n, self.K))
        K= self.K

        # Task 3: implement the EM loop to train the GMM
        # Write your code from here...
        
        # updating log likelihoood
        eps = 1e-8  
        mu = initial_params['mu']
        S = initial_params['S']
        pi = initial_params['pi']
        K = initial_params['K']
        ll = 1
        previous_ll = 0
        log_likelihood = []
        iters = []
        niter = 0
        
        while(np.abs(ll-previous_ll) > eps):       
            previous_ll = self.log_likelihood(pi, mu, S)
            log_likelihood.append(previous_ll)
            r = self.E_step(mu, S, pi)
            mu, S, pi =  self.M_step(mu, r)
            ll = self.log_likelihood(pi, mu, S)

        # ... to here.
        assert(mu.shape == (self.K, self.D) and\
               S.shape  == (self.K, self.D, self.D) and\
               pi.shape == (self.K, 1) and\
               r.shape  == (self.n, self.K))
        return mu, S, pi, r, log_likelihood, iters
    
    def initparams(self,X):
        K= self.K
        D= self.D
        mu= np.random.randint(min(X[:,0]),max(X[:,1]),size=(K,D))
        mu=mu.reshape(K,D)
        pi= np.ones(K)/K
        cov= np.zeros((K,D,D))
        for dim in range(len(cov)):
            np.fill_diagonal(cov[dim],np.std(X,axis=0))   
        params={"mu":mu, "S": cov, "pi":pi}
 
        return params


if __name__ == '__main__':
    np.random.seed(43)

    ##########################
    # You can put your tests here - marking
    # will be based on importing this code and calling
    # specific functions with custom input.
    # Do not write code outside the class definition or
    # this if-block.

In [18]:
# import geolocations
with open ('geolocations.csv',newline='') as csvfile:
    data= list(csv.reader(csvfile))
    data=np.asarray(data)
    data=data.astype(float)

In [19]:
data

array([[  6.14512  ,   1.24327  ],
       [  6.5582   ,   3.3466   ],
       [  5.4646   ,  10.0631   ],
       [ 14.1372   , -16.0754   ],
       [ 35.6177   ,   5.0999   ],
       [  0.2774244,  32.4783873],
       [  5.56508  ,  -0.1601   ],
       [ 15.65611  ,  32.48492  ],
       [ 14.7074454, -17.474397 ],
       [ 14.7765117, -17.3226387],
       [  4.03106  ,   9.73532  ],
       [ 24.713139 ,  46.622186 ],
       [ 15.65792  ,  32.46231  ],
       [ 52.4378   ,  -0.76     ],
       [ 13.441518 ,  -6.268147 ],
       [  4.1058   ,   9.7529   ],
       [ 10.1345   ,  14.7117   ],
       [  5.56     ,  -0.2057   ],
       [ -1.8848   ,  29.7772   ],
       [  3.9389   ,   9.7437   ],
       [  7.4431   ,   3.9022   ],
       [  7.8797   ,   2.7675   ],
       [ 21.54238  ,  39.19797  ],
       [ 10.633    ,  38.783    ],
       [ -1.6166   ,  30.1166   ],
       [  0.2408   ,  32.5523   ],
       [ 15.79255  ,  32.52596  ],
       [  6.531475 ,   3.32405  ],
       [  4.9535   ,

In [20]:
model= GaussianMixtureModel(data,4)
params= model.initparams(data)

In [22]:
mu, S, pi, r,ll_new_mat= model.train(params)

KeyError: 'K'